In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [7]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/tos/atl3/'

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099

def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

def is_jja(month):
    return (month >= 6) & (month <= 8)


path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [2]:

model = ['ACCESS-CM2',
         'ACCESS-ESM1-5', 'BCC-CSM2-MR',
         'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg','GFDL-ESM4',
         'INM-CM4-8', 'INM-CM5-0',
         'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0','NESM3']


In [3]:

period_1_str = 1950
period_1_end = 1999


data_hadi = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/HADI_SST/HadISST_sst.nc')
data_hadi = data_hadi.where(data_hadi.sst < 100)
data_hadi = data_hadi.where(data_hadi.sst >- 100)

data_hadi = Atools.data_sub(data_hadi.sst,-20,0,-3,3)
data_hadi = data_hadi.sel(time=slice(datetime(period_1_str, 1, 1),
                                               datetime(period_1_end, 12, 31)))

In [4]:


sst_hadi_mean = data_hadi.mean(dim='longitude').mean(dim='latitude')

sst_hadi_dtd = nandetrend(sst_hadi_mean)
sst_hadi_dtd_mean = sst_hadi_dtd.detrended.mean(dim='time')
sst_hadi_jja_dtd_mean = sst_hadi_dtd.detrended.sel(time=is_jja(sst_hadi_dtd.detrended['time.month'])).mean(dim='time')

# Historical

In [5]:

sst_bias_jja = np.ones((len(model)))
sst_bias = np.ones((len(model)))


for i in range(0,len(model)):

    sst = xr.open_dataset(dir_hist_sst+'tos_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl3.nc')

    try:

        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        sst['time'] = sst.indexes['time'].to_datetimeindex()
        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    sst = sst.tos 
    sst = sst.mean(dim='lon').mean(dim='lat')
    sst_dtd = nandetrend(sst)
    
    sst_dtd_jja_mean = sst_dtd.detrended.sel(time=is_jja(sst_dtd.detrended['time.month'])).mean(dim='time')
    
    sst_dtd_mean = sst_dtd.detrended.mean(dim='time')
    sst_bias_jja[i] = sst_dtd_jja_mean - sst_hadi_jja_dtd_mean
    sst_bias[i] = sst_dtd_mean- sst_hadi_dtd_mean
    

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  del sys.path[0]
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  del sys.path[0]
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar. 

In [9]:
bias_jja_cmip6  = xr.Dataset({'bias': (['product'],sst_bias_jja),
                       }
                       ,coords={'product':model},attrs={'standard_name': 'JJA SST bias',
                                    'long_name': 'JJA SST bias relative to HadISST 1950-99',
                                    'units': 'K',
                                    'model': 'CMIP6',
                                    'Scenario': 'HIST',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

bias_cmip6  = xr.Dataset({'bias': (['product'],sst_bias),
                       }
                       ,coords={'product':model},attrs={'standard_name': 'SST bias',
                                    'long_name': 'SST bias relative to HadISST 1950-99',
                                    'units': 'K',
                                    'model': 'CMIP6',
                                    'Scenario': 'SSP585',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})



In [10]:

bias_jja_cmip6.to_netcdf(path_data+'sst_bias_atl3_CMIP6_jja_dtd.nc')
bias_cmip6.to_netcdf(path_data+'sst_bias_atl3_CMIP6_dtd.nc')

